# Setup and Definitions

TODO blurb

In [1]:
import sympy as sp
from sympy import sin, cos, tan
from sympy.abc import alpha, psi, sigma, i, d, iota, delta
from analemma.algebra import frame, render, util

Will also want $s$ on the $n$ frame

## Fixed Stars Frame

First, we define the geometric algebra of 3-space and some basis blades from the frame of the "fixed stars"

![Earth's orientation and orbit](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/MainArena.png?token=GHSAT0AAAAAAB73Q3JM6JGFDMRHPPAJKRQ2ZAWFO7Q "Earth's orientation and orbit.").

In [2]:
(e1, e2, e3) = frame.base()

def check_orthnormal(three_frame) -> None:
    """
    Check that a frame of three vectors is orthonormal

    Ensure that the geometric product of the vectors is equal to the pseudoscalar
    """
    v1, v2, v3 = three_frame
    # this is more elegant
    # assert (v1*v2*v3).trigsimp().obj.equals((e1^e2^e3).obj)
    # but this is significantly faster and equivalent
    assert v1|v1 == v2|v2 == v3|v3 == frame.scalar_element(1)
    assert v1|v2 == v1|v3 == v2|v3 == frame.scalar_element(0)

check_orthnormal((e1, e2, e3))

### Earth Frame

Now, let the tilt of the earth's plane (axis) of rotation be $\alpha$ and measure the earth's rotation by $\psi$. Then we can define the Earth frame as follows.

In [3]:
(f1, f2, f3) = frame.planet()
check_orthnormal((f1, f2, f3))
render.print_3frame((f1,f2,f3), "f")

<IPython.core.display.Math object>

The equatorial plane should only depend on the tilt of the Earth's axis of spin $\alpha$, not the angle by which it has rotated relative to the fixed stars $\psi$.

In [4]:
render.print_expression(r"f_1 \wedge f_2", (f1^f2))

<IPython.core.display.Math object>

### Surface Frame

Define an orthonormal frame embedded in the Earth's surface, with $n_1$ pointing South, $n_2$ pointing East and $n_3$ pointing up.

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/SurfaceFrame.png?token=GHSAT0AAAAAAB73Q3JNXNOAJWYLCUINTVLUZAWF6JQ "Frame embedded in Earth's surface.").

<!-- ![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/SurfaceFrame.svg?token=GHSAT0AAAAAAB73Q3JNXNOAJWYLCUINTVLUZAWF6JQ "Frame embedded in Earth's surface."). -->

In [5]:
n1, n2, n3 = frame.surface()

check_orthnormal((n1, n2, n3))

render.print_3frame((n1,n2,n3), "n")

<IPython.core.display.Math object>

### Orbit Rotor and Meridian Plane

Earth orbit rotor $R_\sigma$, and vector parallel to rays of sunshine, $s$.

In [6]:
s = frame.sunray()

render.print_expression("s", s)

<IPython.core.display.Math object>

The meridian plane, $M$.

In [7]:
M = (n1^n3).trigsimp()
render.print_expression("M", M)

<IPython.core.display.Math object>

The noon line is the intersection of the sunshine vector $s$ and the meridian plane $M$, which occurs where $s \wedge M$ vanishes.

In [8]:
render.display( (s^M).trigsimp() )

<IPython.core.display.Math object>

In [9]:
coeff = (s^M).trigsimp().get_coefs(3)[0]
soln = sp.solve(coeff.subs(sin(psi), tan(psi)*cos(psi)), tan(psi))[0]

assert soln.equals(tan(sigma)/cos(alpha))
render.print_expression( r"\tan(\psi)", soln )

<IPython.core.display.Math object>

### Dial face and gnomon

Define an orthonormal frame $u_1, u_2, u_3$ as the unevaluated version of $n_1, n_2, n_3$.

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/DialFrame.png?token=GHSAT0AAAAAAB73Q3JNJN46TIEHP3QCWWGYZAWGADA "Frame embedded in the sundial's face.").


In [10]:
u1, u2, u3 = frame.base("u")
U = u1^u2^u3
display(U)

u_1^u_2^u_3

Frame embedded in dial face.

In [11]:
m1, m2, m3 = frame.dial()

render.print_3frame((m1,m2,m3), "m")

<IPython.core.display.Math object>

Dial face expressed relative to $u$ basis: $G_u$.

In [12]:
Gu = (m1^m2).trigsimp()
render.print_expression( "G_u", Gu)

# check that we get the same result by rotating u1^u2
assert Gu == util.rotate(util.rotate(u1^u2, i, u1^u3), d, u1^u2).trigsimp()


The gnomon expressed relative to the $u$ frame, $g_u$.

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/Gnomon.png?token=GHSAT0AAAAAAB73Q3JNHJMDP6T55SWPTQFGZAWGBNA "The gnomon.").


In [13]:
gu = frame.gnomon()

# extra manipulation to display exactly as in paper
render.print_expression("g_u", sp.collect(sp.trigsimp(gu.obj), -sin(iota)))

<IPython.core.display.Math object>

Projected onto the fixed-stars basis, the gnomon is

In [14]:
g = sum([ c*ni for c, ni in zip(gu.get_coefs(1),[n1, n2, n3])]) # TODO add to analemma.algebra.util?
g = g.trigsimp()

render.print_vector("g", g)

<IPython.core.display.Math object>

The gnomon lies in the meridian plane when the following trivector vanishes:

In [16]:
M_wedge_g = M^g
assert M_wedge_g.obj.trigsimp().equals((sin(delta)*sin(iota)*e1^e2^e3).obj)
render.display( M_wedge_g )

<IPython.core.display.Math object>

Will also want $s$ on the $n$ frame

In [17]:
coeffs = [sp.trigsimp((s|c).obj) for c in [n1, n2, n3]]
su = sum([coeff*vec for coeff, vec in zip(coeffs, [u1, u2, u3])])
render.print_vector("s_u", su, "u")


<IPython.core.display.Math object>